# Determining Equivalence Relative Spectral Response

## Table of Contents

- [Import sensor data](#Import-data)
- [Sensor plots](#Sensor-plots)
- [Solar spectrum](#Solar-spectrum)
- [Synthetic bands](#Synthetic-bands)
- [Hyperspectral](#Hyperspectral)
- [Reference colours](#Reference-colours)
- [Band equivalence](#Band-equivalence)
- [Spectral libraries](#Spectral-libraries)
- [NASA sensor plot emulation](#NASA-sensor-plot-emulation)

# Imports

In [ ]:
import sys, os
import numpy as np
import csv
import xml.etree.ElementTree as ET
from scipy import ndimage
import pandas
from pathlib import Path
import plotly
import zipfile
import netCDF4 
from ipywidgets import interact, Dropdown, FloatSlider, IntSlider, SelectMultiple, Text
from matplotlib import figure
import json

# Import functions from common speclib functions file
sys.path.insert(0, os.path.abspath('../../'))
import speclib

from cryptography.fernet import Fernet

# Import-data

In [ ]:
data_folder = Path("data/")

In [ ]:
# Import Sentinel-3 from NetCDF
s3aolcinc = netCDF4.Dataset(data_folder.joinpath('sentinel3olci/S3A_OL_SRF_20160713_mean_rsr.nc4'))
s3bolcinc = netCDF4.Dataset(data_folder.joinpath('sentinel3olci/S3B_OL_SRF_0_20180109_mean_rsr.nc4'))

In [ ]:
spot1 = pandas.read_excel(data_folder.joinpath('spot/r453_9_spectralsensivity.xls'), sheet_name = 'SPOT 1', header=4, skiprows=[42,43]) #hrv1, hrv2
spot2 = pandas.read_excel(data_folder.joinpath('spot/r453_9_spectralsensivity.xls'), sheet_name = 'SPOT 2', header=4, skiprows=[42,43]) #hrv1, hrv2
spot3 = pandas.read_excel(data_folder.joinpath('spot/r453_9_spectralsensivity.xls'), sheet_name = 'SPOT 3', header=4, skiprows=[42,43]) #hrv1, hrv2
spot4 = pandas.read_excel(data_folder.joinpath('spot/r453_9_spectralsensivity.xls'), sheet_name = 'SPOT 4', header=4, skiprows=[68,69]) #hrvir1, hrvir2
spot5 = pandas.read_excel(data_folder.joinpath('spot/r453_9_spectralsensivity.xls'), sheet_name = 'SPOT 5', header=6) #hrg1, hrg2
qb02 = pandas.read_excel(data_folder.joinpath('quickbird2/QB02_WV01_WV02_Spectral_Response_Final_Correct_Jan292013.xlsx'),\
                               sheet_name = 'QB02 Relative Radiance Response', header=7)
wv01 = pandas.read_excel(data_folder.joinpath('quickbird2/QB02_WV01_WV02_Spectral_Response_Final_Correct_Jan292013.xlsx'),\
                               sheet_name = 'WV01 Relative Radiance Response', header=7)
wv02 = pandas.read_excel(data_folder.joinpath('quickbird2/QB02_WV01_WV02_Spectral_Response_Final_Correct_Jan292013.xlsx'),\
                               sheet_name = 'WV02 Relative Radiance Response', header=7)
geoeye1 = pandas.read_excel(data_folder.joinpath('geoeye1/GeoEye-1_Relative_Spectral_Response.xls'), sheet_name = 'Sheet1', header = 26, nrows= 651)

ls9oli = pandas.read_excel(data_folder.joinpath('landsat9/L9_OLI2_Ball_BA_RSR.v1.0.xlsx'), sheet_name = ['CoastalAerosol','Blue', 'Green', 'Red', 'NIR', 'Cirrus', 'SWIR1', 'SWIR2', 'Pan'])
ls9tirs = pandas.read_excel(data_folder.joinpath('landsat9/L9_TIRS2_Relative_Spectral_Responses.BA.v1.0.xlsx'), sheet_name = ['TIRS Band 10 BA RSR','TIRS Band 11 BA RSR'])

ls8oli = pandas.read_excel(data_folder.joinpath('landsat8/Ball_BA_RSR.xlsx'), sheet_name = ['CoastalAerosol','Blue', 'Green', 'Red', 'NIR', 'Cirrus', 'SWIR1', 'SWIR2', 'Pan'])
ls8tirs = pandas.read_excel(data_folder.joinpath('landsat8/TIRS_Relative_Spectral_ResponsesBA.xlsx'), sheet_name = ['TIRS BA RSR'])

ls7etm = pandas.read_excel(data_folder.joinpath('landsat7/L7_RSR.xlsx'),sheet_name = ['Blue-L7','Green-L7','Red-L7','NIR-L7', 'SWIR(5)-L7','SWIR(7)-L7','Pan-L7'])
ls7band6 = pandas.read_excel(data_folder.joinpath('landsat7/HANDBOOK_LandsatThermalRSRs.spectral.dat.xlsx'), sheet_name = ['HANDBOOK spectral_b6.dat'], header=2, usecols=[0,1])

ls5tm = pandas.read_excel(data_folder.joinpath('landsat5/L5_TM_RSR.xlsx'), sheet_name = ['Blue-L5 TM', 'Green-L5 TM','Red-L5 TM', 'NIR-L5 TM', 'SWIR(5)-L5 TM','SWIR(7)-L5 TM' ] )
ls5band6 = pandas.read_excel(data_folder.joinpath('landsat7/HANDBOOK_LandsatThermalRSRs.spectral.dat.xlsx'), sheet_name = ['HANDBOOK spectral_b6.dat'], header=2, usecols=[0,3])

ls5mss = pandas.read_excel(data_folder.joinpath('landsat5/Landsat5MSS_BandAverageRSR.xlsx'), sheet_name = ['Band 1', 'Band 2', 'Band 3', 'Band 4'], header=1)

ls4tm =  pandas.read_excel(data_folder.joinpath('landsat4/L4_TM_RSR.xlsx'), sheet_name = ['Blue-L4 TM', 'Green-L4 TM','Red-L4 TM', 'NIR-L4 TM', 'SWIR(5)-L4 TM','SWIR(7)-L4 TM' ] )
ls4band6 = pandas.read_excel(data_folder.joinpath('landsat7/HANDBOOK_LandsatThermalRSRs.spectral.dat.xlsx'), sheet_name = ['HANDBOOK spectral_b6.dat'], header=2, usecols=[0,2])

ls4mss = pandas.read_excel(data_folder.joinpath('landsat4/Landsat4MSS_BandAverageRSR.xlsx'), sheet_name = ['Band 1', 'Band 2', 'Band 3', 'Band 4'], header=1) 
ls3mss = pandas.read_excel(data_folder.joinpath('landsat3/Landsat3MSS_BandAverageRSR.xlsx'), sheet_name = ['Band 4', 'Band 5', 'Band 6', 'Band 7'], header=1)
ls2mss = pandas.read_excel(data_folder.joinpath('landsat2/Landsat2MSS_BandAverageRSR.xls'), sheet_name = ['Band 4', 'Band 5', 'Band 6', 'Band 7'], header=1)
ls1mss = pandas.read_excel(data_folder.joinpath('landsat1/Landsat1MSS_BandAverageRSR.xls'), sheet_name = ['Band 4', 'Band 5', 'Band 6', 'Band 7'], header=1)

s2amsi = pandas.read_excel(data_folder.joinpath('sentinel2msi/Sentinel-2 MSI Spectral Responses.xlsx'),sheet_name = ['Spectral Responses (S2A)'])
s2bmsi = pandas.read_excel(data_folder.joinpath('sentinel2msi/Sentinel-2 MSI Spectral Responses.xlsx'),sheet_name = ['Spectral Responses (S2B)'])
tmodis = pandas.read_excel(data_folder.joinpath('aquamodis/MODIS_FM1_IB_OOB_RSR_merged.xls'), sheet_name = ['modis_fm1_rsp_mid_chnl_sum'])
nppviirs = pandas.read_excel(data_folder.joinpath('nppviirs/NG_VIIRS_NPP_RSR_filtered_Oct2011_BA.xlsx'), 
                             sheet_name = ['I1','I2', 'I3','I5', 'M1', 'M2','M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9','M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16A', 'M16B'],
                             header = None)
#AVHRR https://www.star.nesdis.noaa.gov/smcd/spb/fwu/homepage/AVHRR/spec_resp_func/index.html
noaa18avhrr = {}
noaa18avhrr ['1'] = pandas.read_csv(data_folder.joinpath('noaa18avhrr/NOAA_18_A306C001.csv'), usecols=[0,1])
noaa18avhrr ['2'] = pandas.read_csv(data_folder.joinpath('noaa18avhrr/NOAA_18_A306C002.csv'), usecols=[0,1])
noaa18avhrr ['3A'] = pandas.read_csv(data_folder.joinpath('noaa18avhrr/NOAA_18_A306C03A.csv'), usecols=[0,1])
noaa18avhrr ['3B'] = pandas.read_csv(data_folder.joinpath('noaa18avhrr/NOAA_18_A306C03B.csv'), usecols=[0,1])
noaa18avhrr ['4'] = pandas.read_csv(data_folder.joinpath('noaa18avhrr/NOAA_18_A306C004.csv'), usecols=[0,1])
noaa18avhrr ['5'] = pandas.read_csv(data_folder.joinpath('noaa18avhrr/NOAA_18_A306C005.csv'), usecols=[0,1]) 

noaa19avhrr = {}
noaa19avhrr ['1'] = pandas.read_csv(data_folder.joinpath('noaa19avhrr/NOAA_19_A308C001.csv'), usecols=[0,1])
noaa19avhrr ['2'] = pandas.read_csv(data_folder.joinpath('noaa19avhrr/NOAA_19_A308C002.csv'), usecols=[0,1])
noaa19avhrr ['3A'] = pandas.read_csv(data_folder.joinpath('noaa19avhrr/NOAA_19_A308C03A.csv'), usecols=[0,1])
noaa19avhrr ['3B'] = pandas.read_csv(data_folder.joinpath('noaa19avhrr/NOAA_19_A308C03B.csv'), usecols=[0,1])
noaa19avhrr ['4'] = pandas.read_csv(data_folder.joinpath('noaa19avhrr/NOAA_19_A308C004.csv'), usecols=[0,1])
noaa19avhrr ['5'] = pandas.read_csv(data_folder.joinpath('noaa19avhrr/NOAA_19_A308C005.csv'), usecols=[0,1])                    
                    
#ASTER - http://asterweb.jpl.nasa.gov/content/01_mission/03_instrument/archive/swir.txt (vnir.txt, tir.txt)
astervnir = pandas.read_csv(data_folder.joinpath('terraaster/vnir.txt'),delim_whitespace=True, header=3)
asterswir = pandas.read_csv(data_folder.joinpath('terraaster/swir.txt'),delim_whitespace=True, header=3)
astertir = pandas.read_csv(data_folder.joinpath('terraaster/tir.txt'),delim_whitespace=True, header=3)
satellogic = zipfile.ZipFile(data_folder.joinpath('newsat/Satellogic_MS_SRF.zip'), 'r')
'''
newsat = satellogic.open('Newsat8_srf.csv')
newsat8 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat9_srf.csv')
newsat9 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat10_srf.csv')
newsat10 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat11_srf.csv')
newsat11 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat12_srf.csv')
newsat12 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat13_srf.csv')
newsat13 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat14_srf.csv')
newsat14 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat15_srf.csv')
newsat15 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat16_srf.csv')
newsat16 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat17_srf.csv')
newsat17 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat18_srf.csv')
newsat18 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat19_srf.csv')
newsat19 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat20_srf.csv')
newsat20 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat21_srf.csv')
newsat21 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
newsat = satellogic.open('Newsat22_srf.csv')
newsat22 = pandas.read_csv(newsat,delim_whitespace=False, usecols = ['wavelength (nm)', 'blue_srf', 'green_srf', 'red_srf', 'ir_srf'])
'''
#Planet - https://support.planet.com/hc/en-us/articles/360014290293-Do-you-provide-Relative-Spectral-Response-Curves-RSRs-for-your-satellites-
dove = pandas.read_csv(data_folder.joinpath('planet/dove_r.csv'), usecols=[0,1,2,3,4])
planetscope0c0d = pandas.read_csv(data_folder.joinpath('planet/PlanetScope_RSR_SatID_0c_0d.csv'), usecols=[0,1,2,3,4])
planetscope0e = pandas.read_csv(data_folder.joinpath('planet/PlanetScope_RSR_SatID_0e.csv'), usecols=[0,1,2,3,4])
planetscope0f = pandas.read_csv(data_folder.joinpath('planet/PlanetScope_RSR_SatID_0f_10.csv'), usecols=[0,1,2,3,4])
rapideye = pandas.read_csv(data_folder.joinpath('planet/RapidEye_RSR.csv'), usecols=[0,1,2,3,4])
skysat1 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat1.csv'), usecols=[0,1,2,3,4,5])
skysat2 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat2.csv'), usecols=[0,1,2,3,4,5])
skysat3 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat3.csv'), usecols=[0,1,2,3,4,5])
skysat4 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat4.csv'), usecols=[0,1,2,3,4,5])
skysat5 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat5.csv'), usecols=[0,1,2,3,4,5])
skysat6 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat6.csv'), usecols=[0,1,2,3,4,5])
skysat7 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat7.csv'), usecols=[0,1,2,3,4,5])
skysat8 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat8.csv'), usecols=[0,1,2,3,4,5])
skysat9 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat9.csv'), usecols=[0,1,2,3,4,5])
skysat10 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat10.csv'), usecols=[0,1,2,3,4,5])
skysat11 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat11.csv'), usecols=[0,1,2,3,4,5])
skysat12 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat12.csv'), usecols=[0,1,2,3,4,5])
skysat13 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat13.csv'), usecols=[0,1,2,3,4,5])
skysat14_19 = pandas.read_csv(data_folder.joinpath('planet/Skysat_RSR_Skysat14-SkySat19.csv'), usecols=[0,1,2,3,4,5])
superdove = pandas.read_csv(data_folder.joinpath('planet/Superdove.csv'), usecols=[0,1,2,3,4,5,6,7,8])


#H8 - http://www.data.jma.go.jp/mscweb/en/himawari89/space_segment/spsg_ahi.html
h8ahi = pandas.read_excel(data_folder.joinpath('himawari8ahi/AHI-8_SpectralResponsivity_Data.xlsx'), sheet_name = ['Band 1','Band 2','Band 3','Band 4','Band 5','Band 6','Band 7','Band 8','Band 9','Band 10','Band 11','Band 12','Band 13','Band 14','Band 15','Band 16'], header = [4])
h9ahi = pandas.read_excel(data_folder.joinpath('himawari9ahi/AHI-9_SpectralResponsivity_Data.xlsx'), sheet_name = ['Band 1','Band 2','Band 3','Band 4','Band 5','Band 6','Band 7','Band 8','Band 9','Band 10','Band 11','Band 12','Band 13','Band 14','Band 15','Band 16'], header = [4])


irradiance = pandas.read_excel(data_folder.joinpath('solar_irradiance/ASTMG173.xls'), sheet_name = ['SMARTS2'], header=1)
solar_tsis = pandas.read_csv(data_folder.joinpath('solar_irradiance/tsis1_hsrs_p1nm.csv'), usecols = ['wavelength (nm)','irradiance (W/m^2/nm)','uncertainty (W/m^2/nm)'])
transmission = pandas.read_excel(data_folder.joinpath('solar_irradiance/Atmos_constits.xls'), sheet_name = ['solartwo'], header=11, index_col=None, usecols ="A,B")
irradianceWhi2008 = pandas.read_csv(data_folder.joinpath('solar_irradiance/ref_solar_irradiance_whi-2008_ver2.dat.txt'), sep="  ", header=141, index_col=None)

irradianceThuillier2002 = pandas.read_excel(data_folder.joinpath('solar_irradiance/Solar_irradiance_Thuillier_2002.xls'),  sheet_name = ['Thuillier 2002'], index_col=None, usecols ="A,B")


In [ ]:
'''
print ("NewSat 8", newsat8.keys())
'''
print ("Planet Dove", dove.keys())
print ("Planetscope 0x 0d", planetscope0c0d.keys())
print ("Planetscope 0e", planetscope0e.keys())
print ("Planetscope 0f", planetscope0f.keys())
print ("Rapideye", rapideye.keys())
print ("Skysat1", skysat1.keys())
print ("Skysat2", skysat2.keys())
print ("Skysat3", skysat3.keys())
print ("Skysat4", skysat4.keys())
print ("Skysat5", skysat5.keys())
print ("Skysat6", skysat6.keys())
print ("Skysat7", skysat7.keys())
print ("Skysat8", skysat8.keys())
print ("Skysat9", skysat9.keys())
print ("Skysat10", skysat10.keys()) 
print ("Skysat11", skysat11.keys())
print ("Skysat12", skysat12.keys()) 
print ("Skysat13", skysat13.keys())
print ("Skysat14 to 19", skysat14_19.keys()) 
print ("SuperDove", superdove.keys())

print ("SPOT 1", spot1.keys())
print ("SPOT 2", spot2.keys())
print ("SPOT 3", spot3.keys())
print ("SPOT 4", spot4.keys())
print ("SPOT 5", spot5.keys())
print ("Geoeye 1", geoeye1.keys())
print ("Worldview 1", wv01.keys())
print ("Worldview 2", wv02.keys())
print ("Quickbird 2", qb02.keys())
print ("ASTER VNIR", astervnir.keys())
print ("ASTER TIR", astertir.keys())
print ("ASTER SWIR", asterswir.keys())
print ("Himawari 8 AHI", h8ahi.keys())
print ("Himawari 9 AHI", h9ahi.keys())
print ("OLI2 ",ls9oli.keys())
print ("TIRS2", ls9tirs.keys())
print ("OLI ",ls8oli.keys())
print ("TIRS", ls8tirs.keys())
print ("TM", ls5tm.keys())
print ("TM Band6", ls5band6.keys())
print ("ETM+", ls7etm.keys())
print ("MSS",ls1mss.keys())
print ("S2AMSI", s2amsi.keys())
print ("S2BMSI", s2bmsi.keys())
print ("Terra MODIS", (tmodis.keys()))
print ("NPP VIIRS", (nppviirs.keys()))
print ("NOAA18 AVHRR", noaa18avhrr .keys())
print ("NOAA19 AVHRR", noaa19avhrr .keys())
print ("Solar Irradiance SMARTS", (irradiance.keys()))
print ("Solar Irradiance TSIS", (solar_tsis.keys()))
print ("Solar Irradiance Thuillier", (irradianceThuillier2002.keys()))
print ("Solar Irradiance WHI", (irradianceWhi2008.keys()))
print ("Atmospheric Transmission", (transmission.keys()))

sensorlist = [spot1, spot2, spot3, spot4, spot5, geoeye1, wv01, wv02, qb02, astervnir,asterswir,astertir, h8ahi,ls8oli,ls8tirs,ls5tm,ls7etm,s2amsi,tmodis]

In [ ]:
ls8oli = speclib.updatecolumns(ls8oli)
ls9oli = speclib.updatecolumns(ls9oli)

nppviirs = speclib.updatecolumns(nppviirs)

for key in h8ahi.keys():
    h8ahi[key].columns = ['Wavelength', 'Wavenumber', 'RSR']
for key in h9ahi.keys():
    h9ahi[key].columns = ['Wavelength', 'Wavenumber', 'RSR']

for key in noaa18avhrr .keys():
    noaa18avhrr[key].columns = ['Wavelength', 'RSR']
    noaa19avhrr[key].columns = ['Wavelength', 'RSR']
    
spot5.columns = ['Wavelength', 'Pa','B1','B2','B3','MIR','Pa.1','B1.1','B2.1','B3.1','MIR.1']   
geoeye1.columns = ['Wavelength', 'Panchromatic', 'Blue', 'Green', 'Red','Near IR']
'''
newsat8.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat9.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat10.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat11.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat12.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat13.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat14.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat15.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat16.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat17.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat18.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat19.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat20.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat21.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
newsat22.columns = ['Wavelength',  'Blue', 'Green', 'Red','Near IR']
'''
dove.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR'] #nm
planetscope0c0d.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR']
planetscope0e.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR']
planetscope0f.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR']
rapideye.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR']
skysat1.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat2.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat3.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat4.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat5.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat6.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat7.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat8.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat9.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat10.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan'] 
skysat11.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat12.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan'] 
skysat13.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR', 'Pan']
skysat14_19.columns = ['Wavelength', 'Blue', 'Green', 'Red', 'NIR','Pan'] 
superdove.columns = ['Wavelength', 'Coastal-Blue',	'Blue', 'Green', 'Green_ii', 'Yellow', 'Red', 'Red-edge', 'NIR'] #nm

wv01.columns = ['Wavelength', 'Pan']
wv02.columns = ['Wavelength', 'Pan', 'Blue', 'Green', 'Red', 'NIR1', 'Coastal', 'Yellow', 'RedEdge','NIR2']
qb02.columns = ['Wavelength', 'Pan', 'Blue', 'Green', 'Red','NIR']
astervnir.columns = ['Wavelength1','VNIR_Band1','Wavelength2','VNIR_Band2','Wavelength3','VNIR_Band3', 'Wavelength3B','VNIR_Band3B']
asterswir.columns = ['Wavelength4','SWIR_Band4','Wavelength5','SWIR_Band5','Wavelength6','SWIR_Band6','Wavelength7','SWIR_Band7','Wavelength8','SWIR_Band8','Wavelength9','SWIR_Band9']
astertir.columns = ['Wavelength10','TIR_Band10','Wavelength11','TIR_Band11','Wavelength12','TIR_Band12','Wavelength13','TIR_Band13','Wavelength14','TIR_Band14']

ls9tirs['TIRS Band 10 BA RSR'].columns = ['Wavelength', 'Band 10 Band Average RSR', 'ignore', 'ignore']
ls9tirs['TIRS Band 11 BA RSR'].columns = ['Wavelength', 'Band 11 Band Average RSR', 'ignore', 'ignore']
ls8tirs['TIRS BA RSR'].columns = ['Wavelength', 'TIRS1RSR', 'TIRS2RSR', 'ignore','ignore','ignore','ignore']
ls7band6['HANDBOOK spectral_b6.dat'].columns = ['Wavelength', 'Band6']
ls5band6['HANDBOOK spectral_b6.dat'].columns = ['Wavelength', 'Band6']
ls4band6['HANDBOOK spectral_b6.dat'].columns = ['Wavelength', 'Band6']

ls7etm = speclib.updatecolumns( ls7etm)
ls5tm = speclib.updatecolumns( ls5tm)
ls4tm = speclib.updatecolumns( ls4tm)

ls5mss = speclib.updatecolumns( ls5mss)
ls4mss = speclib.updatecolumns( ls4mss)
ls3mss = speclib.updatecolumns( ls3mss)
ls2mss = speclib.updatecolumns( ls2mss)
ls1mss = speclib.updatecolumns( ls1mss)


s2amsi['Spectral Responses (S2A)'].columns = ['Wavelength', 'Band1', 'Band2', 'Band3', 'Band4', 'Band5',
       'Band6', 'Band7', 'Band8', 'Band8A', 'Band9', 'Band10',
       'Band11', 'Band12']
s2bmsi['Spectral Responses (S2B)'].columns = ['Wavelength', 'Band1', 'Band2', 'Band3', 'Band4', 'Band5',
       'Band6', 'Band7', 'Band8', 'Band8A', 'Band9', 'Band10',
       'Band11', 'Band12']
tmodis['modis_fm1_rsp_mid_chnl_sum'].columns = ['Wavelength1', 'Band1RSR', 'Wavelength2', 'Band2RSR', 'Wavelength3',
       'Band3RSR', 'Wavelength4', 'Band4RSR', 'Wavelength5', 'Band5RSR',
       'Wavelength6', 'Band6RSR', 'Wavelength7', 'Band7RSR', 'Wavelength8',
       'Band8RSR', 'Wavelength9', 'Band9RSR', 'Wavelength10', 'Band10RSR',
       'Wavelength11', 'Band11RSR', 'Wavelength12', 'Band12RSR', 'Wavelength13',
       'Band13RSR', 'Wavelength14', 'Band14RSR', 'Wavelength15', 'Band15RSR',
       'Wavelength16', 'Band16RSR', 'Wavelength17', 'Band17RSR', 'Wavelength18',
       'Band18RSR', 'Wavelength19', 'Band19RSR', 'Wavelength20', 'Band20RSR',
       'Wavelength21', 'Band21RSR', 'Wavelength22', 'Band22RSR', 'Wavelength23',
       'Band23RSR', 'Wavelength24', 'Band24RSR', 'Wavelength25', 'Band25RSR',
       'Wavelength26', 'Band26RSR', 'Wavelength27', 'Band27RSR', 'Wavelength28',
       'Band28RSR', 'Wavelength29', 'Band29RSR', 'Wavelength30', 'Band30RSR',
       'Wavelength31', 'Band31RSR', 'Wavelength32', 'Band32RSR', 'Wavelength33',
       'Band33RSR', 'Wavelength34', 'Band34RSR', 'Wavelength35', 'Band35RSR',
       'Wavelength36', 'Band36RSR']

irradiance['SMARTS2'].columns = ['Wavelength', 'ETR', 'Global_Tilt', 'Direct_Circumsolar']

In [ ]:
# Define a sensible dataframe to hold the sensor info - above is getting blurghy
# Room to add sensor native sampling resolution too though we can consider that out of scope for now
# combinedsensors = pandas.DataFrame([{ "wavelength": , "rsr":  }], dtype = 'sensor')
# A dict of pandas frames sounds like a reasonable container for this

sensordict = {}

#TODO def modes/types

# Type 1 - dict of bands i.e. one band per sheet, one wavelength column, one rsr columns
 
sensor = 'ls9oli'
sensordict[sensor] = {}
inputdict = ls9oli

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors
    
sensor = 'ls8oli'
sensordict[sensor] = {}
inputdict = ls8oli

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'ls7etm'
sensordict[sensor] = {}
inputdict = ls7etm

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength)
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'ls5tm'
sensordict[sensor] = {}
inputdict = ls5tm

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors    
 
sensor = 'ls5mss'
sensordict[sensor] = {}
inputdict = ls5mss

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000)
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'ls4tm'
sensordict[sensor] = {}
inputdict = ls4tm

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors    

sensor = 'ls4mss'
sensordict[sensor] = {}
inputdict = ls4mss

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'ls3mss'
sensordict[sensor] = {}
inputdict = ls3mss

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors    

sensor = 'ls2mss'
sensordict[sensor] = {}
inputdict = ls2mss

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors
    
sensor = 'ls1mss'
sensordict[sensor] = {}
inputdict = ls1mss

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

sensor = 'nppviirs'
sensordict[sensor] = {}
inputdict = nppviirs

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors 
    
sensor = 'h8ahi'
sensordict[sensor] = {}
inputdict = h8ahi

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors    

sensor = 'h9ahi'
sensordict[sensor] = {}
inputdict = h9ahi

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors 
    
sensor = 'noaa18avhrr'
sensordict[sensor] = {}
inputdict = noaa18avhrr 

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors 

sensor = 'noaa19avhrr'
sensordict[sensor] = {}
inputdict = noaa19avhrr 

for key in inputdict.keys():
    sensorrsr = pandas.Series(inputdict[(key)].RSR)
    sensorwavelength = pandas.Series(inputdict[(key)].Wavelength*1000 )
    combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})
    sensordict[sensor][(key)] = combinedsensors

   
# Type 2 - single dict i.e. multiple bands per sheet, one wavelength column, multiple rsr columns 
    

sensordict = speclib.multiplebandsonewavelength(sensordict, s2amsi, 's2amsi',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, s2bmsi, 's2bmsi',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, geoeye1, 'geoeye1',1)
'''
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat8, 'newsat8',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat9, 'newsat9',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat10, 'newsat10',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat11, 'newsat11',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat12, 'newsat12',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat13, 'newsat13',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat14, 'newsat14',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat15, 'newsat15',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat16, 'newsat16',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat17, 'newsat17',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat18, 'newsat18',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat19, 'newsat19',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat20, 'newsat20',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat21, 'newsat21',1)
sensordict = speclib.multiplebandsonewavelength(sensordict, newsat22, 'newsat22',1)
'''
sensordict = speclib.multiplebandsonewavelength(sensordict,dove, 'dove',1)
sensordict = speclib.multiplebandsonewavelength(sensordict,planetscope0c0d, 'planetscope0c0d',1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,planetscope0e, 'planetscope0e', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,planetscope0f, 'planetscope0f', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,rapideye, 'rapideye', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat1,'skysat1', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat2,'skysat2', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat3,'skysat3', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat4,'skysat4', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat5,'skysat5', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat6,'skysat6', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat7,'skysat7', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat8,'skysat8', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat9,'skysat9', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat10,'skysat10', 1000) 
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat11,'skysat11', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat12,'skysat12', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat13,'skysat13', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,skysat14_19, 'skysat14_19', 1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,superdove,'superdove', 1)

sensordict = speclib.multiplebandsonewavelength(sensordict,wv01, 'wv01',1000)
sensordict = speclib.multiplebandsonewavelength(sensordict,wv02, 'wv02',1000)
sensordict = speclib.multiplebandsonewavelength(sensordict, qb02,'qb02',1000)
sensordict = speclib.multiplebandsonewavelength(sensordict, spot5, 'spot5',1)

sensordict = speclib.spotdataframe(sensordict, 'spot1',spot1)
sensordict = speclib.spotdataframe(sensordict, 'spot2',spot2)
sensordict = speclib.spotdataframe(sensordict, 'spot3',spot3)
sensordict = speclib.spotdataframe(sensordict, 'spot4',spot4)

In [ ]:
# Type 4 - Solar Irradiance - supposing we normalise this or find a normalised source other than 
# http://www.astm.org/Standards/G173.htm 
sensor = 'irradiance'
sensordict[sensor] = {}
inputdict = irradiance
sensordict[sensor][('solar')] = pandas.DataFrame({'wavelength': \
                                                       inputdict['SMARTS2'].Wavelength,\
                                                       'rsr': \
                                                       inputdict['SMARTS2'].Direct_Circumsolar})
sensordict[sensor][('ETR')] = pandas.DataFrame({'wavelength': \
                                                       inputdict['SMARTS2'].Wavelength,\
                                                       'rsr': \
                                                       inputdict['SMARTS2'].ETR})
sensordict[sensor][('Global_Tilt')] = pandas.DataFrame({'wavelength': \
                                                       inputdict['SMARTS2'].Wavelength,\
                                                       'rsr': \
                                                       inputdict['SMARTS2'].Global_Tilt})

solar_curve = ndimage.filters.gaussian_filter(sensordict['irradiance'][('solar')].rsr, 20)

sensor = 'irradianceThuillier2002'

sensordict[sensor] = {}
inputdict = irradianceThuillier2002
sensordict[sensor][('Thuillier 2002')] = pandas.DataFrame({'wavelength': \
                                                       inputdict['Thuillier 2002'].nm,\
                                                       'rsr': \
                                                       inputdict['Thuillier 2002']['mW/m2/nm']})
solar_curve = ndimage.filters.gaussian_filter(sensordict['irradianceThuillier2002'][('Thuillier 2002')].rsr, 20)

In [ ]:
# Recreation of USGS infographic
# https://landsat.gsfc.nasa.gov/article/the-intervening-atmosphere-tracing-the-provenance-of-a-favorite-landsat-infographic
sensor = 'transmission'
sensordict[sensor] = {}
inputdict = transmission
sensordict[sensor][('solar')] = pandas.DataFrame({'wavelength': \
                                                       transmission['solartwo']['Wave']*1000,\
                                                       'rsr': \
                                                       transmission['solartwo']['Tot Trans']})

In [ ]:
# Solar Irradiance 
# http://www.astm.org/Standards/G173.htm 
sensor = 'solar_tsis'
sensordict[sensor] = {}
inputdict = solar_tsis
sensordict[sensor][('solar')] = pandas.DataFrame({'wavelength': \
                                                       solar_tsis['wavelength (nm)'],\
                                                       'rsr': \
                                                       solar_tsis['irradiance (W/m^2/nm)']})

In [ ]:
# Solar Spectral Irradiance Reference Spectra for Whole Heliosphere Interval (WHI) 2008 Carrington Rotation 2068 20 March 2008  to  16 April 2008
irradianceWhi2008.reset_index(inplace=True)
sensor = 'irradianceWhi2008'
sensordict[sensor] = {}
inputdict = irradianceWhi2008
sensordict[sensor][('Irradiance Mar 25-29')] = pandas.DataFrame({'wavelength': \
                                                       inputdict['index'],\
                                                       'rsr': \
                                                       inputdict['5 columns : :']})
sensordict[sensor][('Irradiance Mar 30-Apr 4')] = pandas.DataFrame({'wavelength': \
                                                       inputdict['index'],\
                                                       'rsr': \
                                                       inputdict['Wavelength (nm),']})
sensordict[sensor][('Irradiance Apr 10-16')] = pandas.DataFrame({'wavelength': \
                                                       inputdict['index'],\
                                                       'rsr': \
                                                       inputdict['Irradiance Mar 25-29,']})

In [ ]:
s3aolcidict = {}
count = 0
while count < 21:
    sensorrsr = pandas.Series(s3aolcinc['mean_spectral_response_function'][count].astype('float').data)
    sensorwavelength = pandas.Series(s3aolcinc['mean_spectral_response_function_wavelength'][count].astype('int64').data)
    s3aolcidict['OA'+str(count+1)] = speclib.trim_zero_records(speclib.fill_nans(pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})))
    count = count+1
sensordict['s3aolci'] = s3aolcidict

In [ ]:
s3bolcidict = {}
count = 0
while count < 21:
    sensorrsr = pandas.Series(s3bolcinc['mean_spectral_response_function'][count].astype('float').data)
    sensorwavelength = pandas.Series(s3bolcinc['mean_spectral_response_function_wavelength'][count].astype('int64').data)
    s3bolcidict['OA'+str(count+1)] = speclib.trim_zero_records(speclib.fill_nans(pandas.DataFrame({'wavelength': sensorwavelength,'rsr': sensorrsr})))
    count = count+1
sensordict['s3bolci'] = s3bolcidict

In [ ]:
sensor = 'ls8tirs'
sensordict[sensor] = {}
inputdict = ls8tirs
for key in inputdict.keys():
    for column in inputdict[(key)].columns:
        if not (column == 'Wavelength') and not (column == 'ignore'):
            sensorrsr = pandas.Series(inputdict[(key)][column])
            sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
            combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength*1000,'rsr': sensorrsr})
            
            sensordict[sensor][(column)] = combinedsensors            

In [ ]:
sensor = 'ls9tirs'
sensordict[sensor] = {}
inputdict = ls9tirs
for key in inputdict.keys():
    for column in inputdict[(key)].columns:
        if not (column == 'Wavelength') and not (column == 'ignore'):
            sensorrsr = pandas.Series(inputdict[(key)][column])
            sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
            combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength*1000,'rsr': sensorrsr})
            
            sensordict[sensor][(column)] = combinedsensors  

In [ ]:
sensor = 'ls7band6'
sensordict[sensor] = {}
inputdict = ls7band6
for key in inputdict.keys():
    for column in inputdict[(key)].columns:
        if not (column == 'Wavelength') and not (column == 'ignore'):
            sensorrsr = pandas.Series(inputdict[(key)][column])
            sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
            combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength*1000,'rsr': sensorrsr})
            
            sensordict[sensor][(column)] = combinedsensors

In [ ]:
sensor = 'ls5band6'
sensordict[sensor] = {}
inputdict = ls5band6
for key in inputdict.keys():
    for column in inputdict[(key)].columns:
        if not (column == 'Wavelength') and not (column == 'ignore'):
            sensorrsr = pandas.Series(inputdict[(key)][column])
            sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
            combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength*1000,'rsr': sensorrsr})
            
            sensordict[sensor][(column)] = combinedsensors

In [ ]:
sensor = 'ls4band6'
sensordict[sensor] = {}
inputdict = ls4band6
for key in inputdict.keys():
    for column in inputdict[(key)].columns:
        if not (column == 'Wavelength') and not (column == 'ignore'):
            sensorrsr = pandas.Series(inputdict[(key)][column])
            sensorwavelength = pandas.Series(inputdict[(key)].Wavelength )
            combinedsensors = pandas.DataFrame({'wavelength': sensorwavelength*1000,'rsr': sensorrsr})
            
            sensordict[sensor][(column)] = combinedsensors

In [ ]:
# Add Thermal Bands to Landsat

In [ ]:
sensordict['ls8olitirs'] = sensordict['ls8oli']
sensordict['ls8olitirs']['TIR1'] = sensordict['ls8tirs']['TIRS1RSR']
sensordict['ls8olitirs']['TIR2'] = sensordict['ls8tirs']['TIRS2RSR']
sensordict['ls9oli2tirs2'] = sensordict['ls9oli']
sensordict['ls9oli2tirs2']['TIR1'] = sensordict['ls9tirs']['Band 10 Band Average RSR']
sensordict['ls9oli2tirs2']['TIR2'] = sensordict['ls9tirs']['Band 11 Band Average RSR']
sensordict['ls7etm']['TIR(6)-L7'] = sensordict['ls7band6']['Band6']
sensordict['ls5tm']['TIR(6)-L5 TM'] = sensordict['ls5band6']['Band6']
sensordict['ls4tm']['TIR(6)-L4 TM'] = sensordict['ls4band6']['Band6']

In [ ]:
# Similar to above - could build an iterator  - some more on this below

sensor = 'tmodis'
sensordict[sensor] = {}
inputdict = tmodis
tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength4*1000
tmodis['modis_fm1_rsp_mid_chnl_sum'].Band4RSR

sensordict[sensor][('Band1')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength1*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band1RSR}))
sensordict[sensor][('Band2')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength2*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band2RSR}))
sensordict[sensor][('Band3')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength3*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band3RSR}))
sensordict[sensor][('Band4')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength4*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band4RSR}))
sensordict[sensor][('Band5')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength5*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band5RSR}))
sensordict[sensor][('Band6')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength6*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band6RSR}))
sensordict[sensor][('Band7')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength7*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band7RSR}))
sensordict[sensor][('Band8')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength8*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band8RSR}))
sensordict[sensor][('Band9')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength9*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band9RSR}))
sensordict[sensor][('Band10')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength10*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band10RSR}))
sensordict[sensor][('Band11')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength11*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band11RSR}))
sensordict[sensor][('Band12')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength12*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band12RSR}))
sensordict[sensor][('Band13')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength13*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band13RSR}))
sensordict[sensor][('Band14')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength14*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band14RSR}))
sensordict[sensor][('Band15')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength15*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band15RSR}))
sensordict[sensor][('Band16')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength16*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band16RSR}))
sensordict[sensor][('Band17')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength17*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band17RSR}))
sensordict[sensor][('Band18')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength18*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band18RSR}))
sensordict[sensor][('Band19')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength19*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band19RSR}))
sensordict[sensor][('Band20')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength20*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band20RSR}))
sensordict[sensor][('Band21')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength21*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band21RSR}))
sensordict[sensor][('Band22')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength22*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band22RSR}))
sensordict[sensor][('Band23')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength23*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band23RSR}))
sensordict[sensor][('Band24')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength24*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band24RSR}))
sensordict[sensor][('Band25')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength25*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band25RSR}))
sensordict[sensor][('Band26')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength26*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band26RSR}))
sensordict[sensor][('Band27')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength27*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band27RSR}))
sensordict[sensor][('Band28')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength28*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band28RSR}))
sensordict[sensor][('Band29')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength29*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band29RSR}))
sensordict[sensor][('Band30')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength30*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band30RSR}))
sensordict[sensor][('Band31')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength31*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band31RSR}))
sensordict[sensor][('Band32')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength32*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band32RSR}))
sensordict[sensor][('Band33')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength33*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band33RSR}))
sensordict[sensor][('Band34')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength34*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band34RSR}))
sensordict[sensor][('Band35')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength35*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band35RSR}))
sensordict[sensor][('Band36')] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Wavelength36*1000,\
                                                       'rsr': \
                                                       tmodis['modis_fm1_rsp_mid_chnl_sum'].Band36RSR}))


In [ ]:
# Type 3 - one band in dict i.e. alternating wavelength and column  per sheet, one wavelength column, one rsr columns
# Combine all of the ASTER telescope to a single dictionary - leaving OLI and TIRS seperate for now

sensor = 'aster'
sensordict[sensor] = {}
inputdict = astervnir
sensordict[sensor][('VNIR_Band1')] = pandas.DataFrame({'wavelength': astervnir.Wavelength1*1000, 'rsr': astervnir.VNIR_Band1})
sensordict[sensor][('VNIR_Band2')] = pandas.DataFrame({'wavelength': astervnir.Wavelength2*1000, 'rsr': astervnir.VNIR_Band2})
sensordict[sensor][('VNIR_Band3')] = pandas.DataFrame({'wavelength': astervnir.Wavelength3*1000, 'rsr': astervnir.VNIR_Band3})
sensordict[sensor][('VNIR_Band3B')] = pandas.DataFrame({'wavelength': astervnir.Wavelength3B*1000, 'rsr': astervnir.VNIR_Band3B})
sensordict[sensor][('SWIR_Band4')] = pandas.DataFrame({'wavelength': asterswir.Wavelength4*1000, 'rsr': asterswir.SWIR_Band4})
sensordict[sensor][('SWIR_Band5')] = pandas.DataFrame({'wavelength': asterswir.Wavelength5*1000, 'rsr': asterswir.SWIR_Band5})
sensordict[sensor][('SWIR_Band6')] = pandas.DataFrame({'wavelength': asterswir.Wavelength6*1000, 'rsr': asterswir.SWIR_Band6})
sensordict[sensor][('SWIR_Band7')] = pandas.DataFrame({'wavelength': asterswir.Wavelength7*1000, 'rsr': asterswir.SWIR_Band7})
sensordict[sensor][('SWIR_Band8')] = pandas.DataFrame({'wavelength': asterswir.Wavelength8*1000, 'rsr': asterswir.SWIR_Band8})
sensordict[sensor][('SWIR_Band9')] = pandas.DataFrame({'wavelength': asterswir.Wavelength9*1000, 'rsr': asterswir.SWIR_Band9})
sensordict[sensor][('TIR_Band10')] = pandas.DataFrame({'wavelength': astertir.Wavelength10*1000, 'rsr': astertir.TIR_Band10})
sensordict[sensor][('TIR_Band11')] = pandas.DataFrame({'wavelength': astertir.Wavelength11*1000, 'rsr': astertir.TIR_Band11})
sensordict[sensor][('TIR_Band12')] = pandas.DataFrame({'wavelength': astertir.Wavelength12*1000, 'rsr': astertir.TIR_Band12})
sensordict[sensor][('TIR_Band13')] = pandas.DataFrame({'wavelength': astertir.Wavelength13*1000, 'rsr': astertir.TIR_Band13})
sensordict[sensor][('TIR_Band14')] = pandas.DataFrame({'wavelength': astertir.Wavelength14*1000, 'rsr': astertir.TIR_Band14})

In [ ]:
# Eliminate NaN wavelength records
for key in sensordict['wv02']:
    sensordict['wv02'][key] = sensordict['wv02'][key][sensordict['wv02'][(key)]['wavelength'].notnull()]

In [ ]:
# Eliminate NaN wavelength records
for key in sensordict['tmodis']:
    sensordict['tmodis'][key] = sensordict['tmodis'][key][sensordict['tmodis'][(key)]['wavelength'].notnull()]

In [ ]:
# Eliminate NaN rsr records
for key in sensordict['spot5']:
    sensordict['spot5'][key] = sensordict['spot5'][key][sensordict['spot5'][(key)]['rsr'].notnull()]

In [ ]:
for sensor in sensordict:
    for band in sensordict[sensor]:
        sensordict[sensor][band] = speclib.trim_zero_records(speclib.fill_nans(sensordict[sensor][band]))

In [ ]:
# The holy grail awaits...
sensordict.keys()
#sensordict[sensor][(column)]

In [ ]:
# Validation of common structure
for sensor in sensordict.keys():
    for band in sensordict[sensor]:
        try:
            left, right = speclib.fwhm_gaussian(sensordict, sensor, band)
            print(sensor, left,right, band, len(sensordict[sensor][band].rsr))
        except:
            print(sensor, band, len(sensordict[sensor][band].rsr))

# Solar-spectrum

In [ ]:
# Plot solar spectrum
# http://gsics.atmos.umd.edu/bin/view/Development/ReferenceSolarSpectrum
# -- 1. TSIS-1 Reference Solar. https://lasp.colorado.edu/lisird/data/tsis1_hsrs_p1nm
# -- 2. Whole Heliosphere Interval spectrum https://lasp.colorado.edu/lisird/resources/whi_ref_spectra/data/ref_solar_irradiance_whi-2008_ver2.dat
# -- 3. SAO2010 spectrum ttps ://www.cfa.harvard.edu/atmosphere/links/sao2010.solref.converted
# -- 4. Solar Radiation Physical Modeling (SRPM) http ://lasp.colorado.edu/lisird/srpm/
# -- 5. SCIAMACHY solar spectrum is in preparation. ??
# -- 6. SOLSPEC solar spectrum ??
# -- 7. Aura/OMI, SORCE SIM, SORCE SOLTICE solar spectrum comparison. ??

# http://rredc.nrel.gov/solar/spectra/am1.5/

solar_curve_normal = (solar_curve - solar_curve.min())/(solar_curve.max() - solar_curve.min())
                            
plotly.offline.iplot({"data": [
    {"x": sensordict['irradiance'][('solar')].wavelength,\
          "y": sensordict['irradiance'][('solar')].rsr/sensordict['irradiance'][('solar')].rsr.max(),\
          "name": 'Solar Irradiance SMARTS2'},
            
    {"x": sensordict['solar_tsis'][('solar')].wavelength,\
          "y": sensordict['solar_tsis'][('solar')].rsr/sensordict['solar_tsis'][('solar')].rsr.max(),\
          "name": 'Solar Irradiance TSIS'}, 

    {"x": sensordict['irradianceWhi2008'][('Irradiance Mar 25-29')].wavelength,\
          "y": sensordict['irradianceWhi2008'][('Irradiance Mar 25-29')].rsr/sensordict['irradianceWhi2008'][('Irradiance Mar 25-29')].rsr.max(),\
          "name": 'Solar Irradiance WHI 2008'}, 

    {"x": sensordict['irradianceThuillier2002'][('Thuillier 2002')].wavelength,\
          "y": sensordict['irradianceThuillier2002'][('Thuillier 2002')].rsr/sensordict['irradianceThuillier2002'][('Thuillier 2002')].rsr.max(),\
          "name": 'Solar Irradiance Thuillier 2002'}, 

 ],"layout": {"title": "Solar Irradiance"}})



In [ ]:
# Example mixing sensordict and individual dict of dataframes access
# We want a function that enables return "sensors that are green" for example

print (plotly.__version__)            # version 1.9.4 required

plotly.offline.iplot({
"data": [{"x": ls9oli[('Green')].Wavelength,"y": ls9oli[('Green')].RSR, "name": 'ls9oli GREEN'},\
         {"x": ls8oli[('Green')].Wavelength,"y": ls8oli[('Green')].RSR, "name": 'ls8oli GREEN'},\
         {"x": ls5tm[('Green-L5 TM')].Wavelength,"y": ls5tm[('Green-L5 TM')].RSR, "name": 'ls5tm GREEN'},\
         {"x": ls7etm[('Green-L7')].Wavelength,"y": ls7etm[('Green-L7')].RSR, "name": 'ls7etm GREEN'},\
         {"x": sensordict['s2amsi'][('Band3')].wavelength,"y": sensordict['s2amsi'][('Band3')].rsr,\
          "name":'s2amsi Band3'},\
         {"x": sensordict['tmodis'][('Band4')].wavelength,"y": sensordict['tmodis'][('Band4')].rsr,\
          "name":'tmodis Band4'},\
         {"x": sensordict['h8ahi'][('Band 2')].wavelength,"y": sensordict['h8ahi'][('Band 2')].rsr,\
          "name": 'h8ahi Band 2'},\
         {"x": sensordict['aster'][('VNIR_Band1')].wavelength,"y": sensordict['aster'][('VNIR_Band1')].rsr, \
          "name": 'astervnir VNIR1'},\
         {"x": sensordict['qb02'][('Green')].wavelength,"y": sensordict['qb02'][('Green')].rsr, \
          "name": 'quickbird 2 Green'},
         {"x": sensordict['irradiance'][('solar')].wavelength,\
          "y": sensordict['irradiance'][('solar')].rsr/sensordict['irradiance'][('solar')].rsr.max(),\
          "name": 'Solar Irradiance'},
        ],
"layout": {"title": "GREEN Relative Spectral Response"}})


# Synthetic-bands 

## Synthetic band creation from nominal centre and fwhm

In [ ]:
# Using typical summary sensor info, manufacture the synthetic rsr to compare with measured relative spectral response
ls8oli_summary = pandas.read_csv(data_folder.joinpath('landsat8/l8oli_summary.txt'))
lnext_summary = pandas.read_csv(data_folder.joinpath('landsatnext/lnext.csv'))
eo1_hyperion_summary = pandas.read_csv(data_folder.joinpath('eo1hyperion/Hyperion_cen_fwhm_av.dat'), header=1)
enmap_vnir = pandas.read_excel(data_folder.joinpath('enmaphsi/EnMAP_Spectral_Bands_update.xlsx'), sheet_name = 'VNIR', header=0 )
enmap_swir = pandas.read_excel(data_folder.joinpath('enmaphsi/EnMAP_Spectral_Bands_update.xlsx'), sheet_name = 'SWIR', header=0 )
enmap = pandas.concat([enmap_vnir, enmap_swir])

In [ ]:
lnext_summary

In [ ]:
sensor = 'synth_lnext'
sensordict[sensor] = {}
wavelength = np.linspace(1,13000, 2500, dtype='int64')

for band, row in lnext_summary.iterrows():
    fwhm = lnext_summary.FWHM[band]
    rsr = speclib.synthetic_rsr(wavelength,lnext_summary.Center[band],fwhm)
    sensordict[sensor][(lnext_summary.Name[band])] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
    sensordict[sensor][(lnext_summary.Name[band])] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})

speclib.plotallbands(sensordict,'synth_lnext')


In [ ]:
sensor = 'synth_ls8oli'
sensordict[sensor] = {}
wavelength = np.linspace(1,2500, 2500, dtype='int64')

for band, row in ls8oli_summary.iterrows():
    fwhm = ls8oli_summary.FWHM_upper[band] - ls8oli_summary.FWHM_lower[band]
    rsr = speclib.synthetic_rsr(wavelength,ls8oli_summary.Central_wavelength[band],fwhm)
    sensordict[sensor][(ls8oli_summary.Band[band])] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))

In [ ]:
# https://earth.esa.int/eogateway/missions/resourcesat-2
# https://earth.esa.int/eogateway/missions/irs-p6-resourcesat-1#instruments-section
# TODO replace with actual RSR

sensor = 'synth_irs_p6r2_liss3'
sensordict[sensor] = {}
wavelength = np.linspace(1,2500, 2500, dtype='int64')
rsr = speclib.synthetic_rsr(wavelength,(520+590)/2, 590-520)
sensordict[sensor]['B2'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,(620+680)/2, 680-620)
sensordict[sensor]['B3'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,(770+860)/2, 860-770)
sensordict[sensor]['B4'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,(1550+1700)/2, 1700-1550)
sensordict[sensor]['B5'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))

sensor = 'synth_irs_p6r2_liss4'
sensordict[sensor] = {}
wavelength = np.linspace(1,2500, 2500, dtype='int64')
rsr = speclib.synthetic_rsr(wavelength,(520+590)/2, 590-520)
sensordict[sensor]['B2'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,(620+680)/2, 680-620)
sensordict[sensor]['B3'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,(770+860)/2, 860-770)
sensordict[sensor]['B4'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))

sensor = 'synth_irs_p6r2_awifs'
sensordict[sensor] = {}
wavelength = np.linspace(1,2500, 2500, dtype='int64')
rsr = speclib.synthetic_rsr(wavelength,(520+590)/2, 590-520)
sensordict[sensor]['B2'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,(620+680)/2, 680-620)
sensordict[sensor]['B3'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,(770+860)/2, 860-770)
sensordict[sensor]['B4'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,(1550+1700)/2, 1700-1550)
sensordict[sensor]['B5'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))


In [ ]:
# Sentinel-2 NG from https://earth.esa.int/living-planet-symposium-2022-presentations/26.05.Thursday/Geneva/1040-1220/02_Celesti.pdf
sentinel2ng = pandas.read_excel(data_folder.joinpath('sentinel2ng/sentinel2ng.xlsx'),skiprows=[0])
sentinel2ng['Band'] = sentinel2ng['Band'].astype('str')
sentinel2ng = sentinel2ng.rename(columns={"Center Wavelength":"cwl", "Bandwidth":"fwhm"})
sensor = 'synth_sentinel2ng'
sensordict[sensor] = {}
wavelength = np.linspace(1,2500, 2500, dtype='int64')
for band, row in sentinel2ng.iterrows():
    fwhm = sentinel2ng.fwhm[band]
    rsr = speclib.synthetic_rsr(wavelength,sentinel2ng.cwl[band],fwhm)
    sensordict[sensor][(sentinel2ng.Band[band])] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
    sensordict[sensor][(sentinel2ng.Band[band])] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
speclib.plotallbands(sensordict,'synth_sentinel2ng')

In [ ]:
# TRISHA, LSTM, SBG, CHIME
chime = pandas.read_excel(data_folder.joinpath('chime/chime.xlsx'),skiprows=[3,4])
chime = chime.transpose().reset_index()
chime = chime.rename(columns={"index": "Band", 0: "cwl", 1: "fwhm"})
chime['cwl'] = chime['cwl'].astype('float64')

sensor = 'synth_chime'
sensordict[sensor] = {}
wavelength = np.linspace(350,2600, 2600, dtype='int64')
for band, row in chime.iterrows():
    rsr = speclib.synthetic_rsr(wavelength,chime.cwl[band],chime.fwhm[band])
    sensordict[sensor][str((chime.Band[band]))] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
speclib.plotallbands(sensordict,'synth_chime')

sbg_vswir = pandas.read_excel(data_folder.joinpath('sbg/sbg.xlsx'), sheet_name='VSWIR',skiprows=[3,4])
sbg_vswir = sbg_vswir.transpose().reset_index()
sbg_vswir = sbg_vswir.rename(columns={"index": "Band", 0: "cwl", 1: "fwhm"})
sbg_vswir['cwl'] = sbg_vswir['cwl'].astype('float64')
sbg_vswir['Band']= sbg_vswir['Band'].astype('str')

sensor = 'synth_sbg_vswir'
sensordict[sensor] = {}
wavelength = np.linspace(350,2600, 2600, dtype='int64')
for band, row in sbg_vswir.iterrows():
    rsr = speclib.synthetic_rsr(wavelength,sbg_vswir.cwl[band],sbg_vswir.fwhm[band])
    sensordict[sensor][str((sbg_vswir.Band[band]))] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
speclib.plotallbands(sensordict,'synth_sbg_vswir')

trishna = pandas.read_excel(data_folder.joinpath('trishna/trishna.xlsx'),skiprows=[3,4])
trishna = trishna.transpose().reset_index()
trishna = trishna.rename(columns={"index": "Band", 0: "cwl", 1: "fwhm"})
trishna['cwl'] = trishna['cwl'].astype('float64')
sensor = 'synth_trishna'
sensordict[sensor] = {}
wavelength = np.linspace(1,14000, 2500, dtype='int64')
for band, row in trishna.iterrows():
    fwhm = trishna.fwhm[band]
    rsr = speclib.synthetic_rsr(wavelength,trishna.cwl[band],fwhm)
    sensordict[sensor][(trishna.Band[band])] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
    sensordict[sensor][(trishna.Band[band])] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
speclib.plotallbands(sensordict,'synth_trishna')

lstm = pandas.read_excel(data_folder.joinpath('lstm/lstm.xlsx'),skiprows=[3,4])
lstm = lstm.transpose().reset_index()
lstm = lstm.rename(columns={"index": "Band", 0: "cwl", 1: "fwhm"})
lstm['cwl'] = lstm['cwl'].astype('float64')
sensor = 'synth_lstm'
sensordict[sensor] = {}
for band, row in lstm.iterrows():
    fwhm = lstm.fwhm[band]
    rsr = speclib.synthetic_rsr(wavelength,lstm.cwl[band],fwhm)
    sensordict[sensor][(lstm.Band[band])] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
    sensordict[sensor][(lstm.Band[band])] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
speclib.plotallbands(sensordict,'synth_lstm')

sbg_tir = pandas.read_excel(data_folder.joinpath('sbg/sbg.xlsx'), sheet_name='TIR', skiprows=[0,1])
sbg_tir = sbg_tir.rename(columns={"Band#": "Band", " Center_Wavelength(µm)":"cwl", " Spectral_Width(FWHM)(nm)":"fwhm"})
sbg_tir['Band#']= sbg_tir['Band'].astype('str')
sbg_tir['cwl'] = sbg_tir.cwl*1000
sensor = 'synth_sbg_tir'
sensordict[sensor] = {}
wavelength = np.linspace(3500,14000, 2500, dtype='int64')
for band, row in sbg_tir.iterrows():
    fwhm = sbg_tir.fwhm[band]
    rsr = speclib.synthetic_rsr(wavelength,sbg_tir.cwl[band],fwhm)
    sensordict[sensor][(sbg_tir.Band[band])] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
    sensordict[sensor][(sbg_tir.Band[band])] = pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr})
speclib.plotallbands(sensordict,'synth_sbg_tir')



In [ ]:
sbg_vswir['Band'][0]

In [ ]:
# Order sensordict to improve ease of selection and plot order
newsensordict={}
for key in sorted(list(sensordict.keys())):
    newsensordict[key] = sensordict[key]
sensordict = newsensordict

# Hyperspectral

## Hyperspectral example with EnMap

In [ ]:
enmapvnir = pandas.read_excel(data_folder.joinpath('enmaphsi/EnMap_Spectral_Bands_update.xlsx'), sheet_name = 'VNIR', dtype = {'BAND #': str, 'CW (nm)': float, 'FWHM(nm)': float})
enmapswir = pandas.read_excel(data_folder.joinpath('enmaphsi/EnMap_Spectral_Bands_update.xlsx'), sheet_name = 'SWIR', dtype = {'BAND #': str, 'CW (nm)': float, 'FWHM(nm)': float})

In [ ]:
for index, row in enmapvnir.iterrows():
    enmapvnir.loc[index, ['BAND #']] = ('VNIR' + enmapvnir.loc[index, ['BAND #']])

In [ ]:
for index, row in enmapswir.iterrows():
    enmapswir.loc[index, ['BAND #']] = ('SWIR' + enmapswir.loc[index, ['BAND #']])

In [ ]:
enmap = pandas.concat([enmapvnir,enmapswir])

In [ ]:
enmap = enmap.rename(columns={"BAND #": "band", "CW (nm)": "cwl", "FWHM (nm)": "fwhm"})

In [ ]:
enmap.reset_index(inplace=True)

In [ ]:
sensor = 'enmap'
sensordict[sensor] = {}
wavelength = np.linspace(1,2700, 2700, dtype='int64')

for band, row in enmap.iterrows():
    rsr = speclib.synthetic_rsr(wavelength,enmap.cwl[band],enmap.fwhm[band])
    sensordict[sensor][str((enmap.band[band]))] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
speclib.plotallbands(sensordict, 'enmap')

## Hyperspectral example with EMIT

In [ ]:
emit = pandas.read_csv(data_folder.joinpath('emit/emit.csv'),delim_whitespace=False)#, dtype = {'col1': str, 'col2': float, 'col3': int}))

In [ ]:
emit = emit.transpose().reset_index()

In [ ]:
emit = emit.rename(columns={"index": "cwl", 0: "fwhm", 1: "good"})
emit['Band'] = emit.index+1

In [ ]:
emit['cwl'] = emit['cwl'].astype('float64')

In [ ]:
sensor = 'emit'
sensordict[sensor] = {}
wavelength = np.linspace(350,2600, 2600, dtype='int64')

for band, row in emit.iterrows():
    rsr = speclib.synthetic_rsr(wavelength,emit.cwl[band],emit.fwhm[band])
    sensordict[sensor][str((emit.Band[band]))] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
speclib.plotallbands(sensordict,'emit')

## Hyperspectral example with PRISMA

In [ ]:
prisma = pandas.read_csv(data_folder.joinpath('prismahyc/prisma.csv'),delim_whitespace=False)#, dtype = {'col1': str, 'col2': float, 'col3': int}))

In [ ]:
prisma = prisma.transpose().reset_index()

In [ ]:
prisma = prisma.rename(columns={"index": "cwl", 0: "fwhm", 1: "good"})
prisma['Band'] = prisma.index+1

In [ ]:
prisma['cwl'] = prisma['cwl'].astype('float64')

In [ ]:
sensor = 'prisma'
sensordict[sensor] = {}
wavelength = np.linspace(350,2600, 2600, dtype='int64')

for band, row in prisma.iterrows():
    rsr = speclib.synthetic_rsr(wavelength,prisma.cwl[band],prisma.fwhm[band])
    sensordict[sensor][str((prisma.Band[band]))] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
speclib.plotallbands(sensordict, 'prisma')

## Hyperspectral example with DESIS

In [ ]:
# XML from DESIS 
tree = ET.parse(data_folder.joinpath('desis/DESIS-HSI-V0220-METADATA.xml'))
root = tree.getroot()


In [ ]:
dfDESIS = pandas.DataFrame()
dftemp = pandas.DataFrame()
dictDESIS = {}
tempdict = {}
#intialise dataframe
count = 0
for child in root[3][11]:
    for band in child:
        if count == 0: 
            dfDESIS[band.tag] = [band.text]
        else:
            dftemp[band.tag] = [band.text]
    if count != 0:
        dfDESIS = pandas.concat([dfDESIS, dftemp])
        dictDESIS[str(count)] = dftemp
    count = count+1 

In [ ]:
dfDESIS.reset_index(inplace=True)

In [ ]:
for index, row in dfDESIS.iterrows():
    dfDESIS.at[index,'wavelengthCenterOfBand'] = float(row['wavelengthCenterOfBand'])
    dfDESIS.at[index,'wavelengthWidthOfBand'] = float(row['wavelengthWidthOfBand'])

In [ ]:
sensor = 'desis'
sensordict[sensor] = {}
wavelength = np.linspace(350,1100, 1100, dtype='int64')

for band, row in dfDESIS.iterrows():
    rsr = speclib.synthetic_rsr(wavelength,dfDESIS.wavelengthCenterOfBand[band],dfDESIS.wavelengthWidthOfBand[band])
    sensordict[sensor][str((dfDESIS.bandNumber[band]))] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
speclib.plotallbands(sensordict, 'desis')

## Hyperspectral example with EO-1 Hyperion

In [ ]:
sensor = 'synth_eo1_hyperion'
sensordict[sensor] = {}
wavelength = np.linspace(1,2700, 2700, dtype='int64')

for band, row in eo1_hyperion_summary.iterrows():
    rsr = speclib.synthetic_rsr(wavelength,eo1_hyperion_summary.cwl[band],eo1_hyperion_summary.fwhm[band])
    sensordict[sensor][str((eo1_hyperion_summary.Band[band]))] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
speclib.plotallbands(sensordict, 'synth_eo1_hyperion')

# Reference-colours

## User reference band for matching

In [ ]:
# https://en.wikipedia.org/wiki/Blue 450–495 nm
# https://en.wikipedia.org/wiki/Green 495–570 nm
# https://en.wikipedia.org/wiki/Red 620–740 nm
# https://en.wikipedia.org/wiki/Infrared 750–1400 nm
sensor = 'user_reference'
sensordict[sensor] = {}

wavelength = np.linspace(1,2500, 2500, dtype='int64')
rsr = speclib.synthetic_rsr(wavelength,478,45)
sensordict[sensor]['BLUE'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,533,75)
sensordict[sensor]['GREEN'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,680,120)
sensordict[sensor]['RED'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))
rsr = speclib.synthetic_rsr(wavelength,1075,651)
sensordict[sensor]['NIR'] = speclib.trim_zero_records(pandas.DataFrame({'wavelength': wavelength, 'rsr': rsr}))

speclib.plotallbands(sensordict, 'user_reference')

# Sensor-plots

In [ ]:
sensorlistW = SelectMultiple(options = sensordict.keys())
startW = Dropdown(options = ['Pause', 'Go'])
titleW = Text(
    value='Label your plot',
    placeholder='Plot label',
    description='String:',
    disabled=False   
)

@interact(sensorlist = sensorlistW, title=titleW, start = startW)

def print_sample(title, sensorlist, start):
    sensorlistW = list(sensordict.keys())
    titleW = title
    startW.options
    if start == 'Go':
        speclib.plotallbandslist(sensordict, sensorlist, title)
        

# Band-equivalence

## Compute equivalence metrics for target and reference spectra

In [ ]:
referenceW = SelectMultiple(options = sensordict.keys())
targetW = SelectMultiple(options = sensordict.keys())
startW = Dropdown(options = ['Pause', 'Go'])
pearson = 0.3

@interact(reference = referenceW, target = targetW, start = startW)

def print_sample(reference, target, start):
    referenceW.options = sensordict.keys()
    targetW.options = sensordict.keys()
    startW.options
    if start == 'Go':
        global spectrumcomparison
        spectrumcomparison = speclib.equivalence(sensordict,reference,target,pearson)
    return(reference, target, start)


## Summarize the equivalence metrics

In [ ]:
try:
    spectrumcomparison = spectrumcomparison.sort_values('weightedcentredelta', ascending=True)
    display(spectrumcomparison)
except:
    print('spectrumcomparison not computed - go to previous step')

## Set the matching threshold here for the equivalence metrics

In [ ]:
style = {'description_width': 'initial'}
pcorrelationW = FloatSlider(
    value=0.6,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Pearsons correlation:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    tooltip='Pearsons correlation coefficient minimum threshold',
    style=style
)

distanceW = FloatSlider(
    value=20.0,
    min=0.0,
    max=1,
    step=0.01,
    description='Earthmover distance:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style
)

weightedcentredeltaW = IntSlider(
    value=70,
    min=0,
    max=500,
    step=5,
    description='Weighted centre delta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style
)

areadeltaW = IntSlider(
    value=70,
    min=0,
    max=500,
    step=5,
    description='Area delta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style
)

fwhmdeltaW = IntSlider(
    value=70,
    min=0,
    max=500,
    step=5,
    description='FWHM Delta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style
)

startW = Dropdown(options = ['Pause', 'Go'])

@interact(pcorrelation = pcorrelationW, distance = distanceW, weightedcentredelta = weightedcentredeltaW, areadelta = areadeltaW, fwhmdelta = fwhmdeltaW, start = startW)

def print_settings(pcorrelation, distance, weightedcentredelta, areadelta, fwhmdelta, start):
    pcorrelationW = pcorrelation
    distanceW = distance
    weightedcentredeltaW = weightedcentredelta
    areadeltaW = areadelta
    fwhmdeltaW = fwhmdelta
    startW.options
    if start == 'Go':    
        
        display(spectrumcomparison[(spectrumcomparison.pcorrelation > pcorrelationW) &\
                   (spectrumcomparison.distance < distanceW) &\
                   (spectrumcomparison.weightedcentredelta < weightedcentredeltaW) &\
                   (spectrumcomparison.areadelta < areadeltaW) &\
                   (spectrumcomparison.areadelta < fwhmdeltaW)
                   ])
    try:
        print("Reference/Sensor1: "+referenceW.value[0], "Target/Sensor2: "+targetW.value[0], pcorrelation, distance, weightedcentredelta, areadelta, fwhmdelta)
    except:
        print("Select a Reference and Target sensor from the 'Compute equivalence metrics for target and reference spectra' dialogue above")

### Preserve the sensordict as a pickle for use elsewhere

In [ ]:
import pickle

with open('sensordict.pickle', 'wb') as handle:
    pickle.dump(sensordict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Create an encryption key

In [ ]:
# Generate the key to unlock the encrypted file
key = Fernet.generate_key()

# string the key in a file
with open('filekey.key', 'wb') as filekey:
    filekey.write(key)
# opening the key
with open('filekey.key', 'rb') as filekey:
    key = filekey.read()

#### Encrypt the pickle file to make it secure

In [ ]:
# Encrypt the pickled file using the encrption key.

# opening the key
with open('filekey.key', 'rb') as filekey:
    key = filekey.read()
# using the key
fernet = Fernet(key)

# opening the original file to encrypt
with open('sensordict.pickle', 'rb') as file:
    original = file.read()

# encrypting the file
encrypted = fernet.encrypt(original)

# opening the file in write mode and

# writing the encrypted data
with open('sensordict.pickle', 'wb') as encrypted_file:
    encrypted_file.write(encrypted)

# The pickle is now encrypted

In [ ]:
# http://www.eoc.csiro.au/hswww/oz_pi/docs/fwhm_notes.pdf From David Jupp
# P.M.  Teillet,  G.  Fedosejevs,  K.J.  Thome,  and  J.L.  Barker,  “Impacts  of  spectral  band  difference  effects  on  radiometric  
#cross-calibration between satellite sensors in the solar-reflective spectral domain,” 
#Remote Sensing of Environment
#, 110, pp. 
#393–409, 2007
# http://stackoverflow.com/questions/10623448/making-gaussians-constrained-by-the-fwhm

In [ ]:
# Solar irradiance curve - how much radiant energy is hitting the sensor...proportionally? 
# http://rredc.nrel.gov/solar/spectra/am1.5/
# TODO - done above but difficult to track down the normalised transmission data for a representative atmosphere
# across the 300 to 15000 nanometer spectrum

In [ ]:
# TODO Band resampling to provide proportional inclusion of multiple summed bands in synthetic band (i.e. with pan band,
# or hyperspectral, narrow bands to broad band)

# Spectral-libraries

## USGS Spectral Library Version 7
https://pubs.er.usgs.gov/publication/ds1035
- Chapter 1: M = Minerals.
- Chapter 2: S = Soils (including rocks, and mineral mixtures).
- Chapter 3: C = Coatings.
- Chapter 4: L = Liquids (including mixtures of liquids, water and other volatiles, and frozen volatiles).
- Chapter 5: O = Organics (including biochemical constituents of plants and chemical compounds). This chapter includes mixtures of organic compounds with basalt as well as the reference basalt. The spectra of the mixtures with basalt show the fundamental absorptions of the organic compounds, which can be saturated in the spectra of the pure substances.
- Chapter 6: A = Artificial (manmade materials, including construction materials, fabrics, manufactured chemicals, processed materials, paint pigments, plastics, and materials introduced into the environment by human activity).
- Chapter 7: V = Vegetation (including biological materials, plant components such as leaves, flowers, and bark, vegetated areas having more than one species present, lichens, biological soil crusts, and mixtures with vegetation).

In [ ]:
# Download from here https://www.sciencebase.gov/catalog/item/5807a2a2e4b0841e59e3a18d to "spectra/."
usgs_splib07 = zipfile.ZipFile(data_folder.joinpath('spectra/usgs_splib07.zip'), 'r')

In [ ]:
txtfiles = [item for item in usgs_splib07.namelist() if 'txt' in item]

In [ ]:
# Clean, cubic spline interpolated spectra
splib07b = [item for item in txtfiles if 'splib07b/' in item]

In [ ]:
# Spectra obtained with a range of sensors
wavelength_asdfr = pandas.read_csv(usgs_splib07.open(([item for item in splib07b if 'ASDFR_0.35-2.5microns_2151ch' in item])[0]))*1000
wavelength_aviris = pandas.read_csv(usgs_splib07.open(([item for item in splib07b if 'AVIRIS_1996_interp_to_2203ch' in item])[0]))*1000
wavelength_beck = pandas.read_csv(usgs_splib07.open(([item for item in splib07b if 'BECK_Beckman_interp._3961_ch' in item])[0]))*1000
wavelength_nic4 = pandas.read_csv(usgs_splib07.open(([item for item in splib07b if 'NIC4_Nicolet_1.12-216microns' in item])[0]))*1000


## List Chapter contents

In [ ]:
minerals_files = [item for item in splib07b if 'ChapterM_' in item]

In [ ]:
soils_files = [item for item in splib07b if 'ChapterS_' in item]

In [ ]:
coatings_files = [item for item in splib07b if 'ChapterC_' in item]

In [ ]:
liquids_files = [item for item in splib07b if 'ChapterL_' in item]

In [ ]:
organics_files = [item for item in splib07b if 'ChapterO_' in item]

In [ ]:
artificial_files = [item for item in splib07b if 'ChapterA_' in item]

In [ ]:
vegetation_files = [item for item in splib07b if 'ChapterV_' in item]

In [ ]:
# Concatenate all files
samplefiles = minerals_files + soils_files + coatings_files + liquids_files + organics_files + artificial_files + vegetation_files

In [ ]:
chapterfiles = {'minerals': minerals_files, 'soils': soils_files, 'coatings': coatings_files, 'liquids': liquids_files, 'organics': organics_files, 'artificial': artificial_files, 'vegetation': vegetation_files }

In [ ]:
# Create a dict of dataframes representing the wavelength and response of the sample spectra 
instrument_list = []
sampledict = {}

for chapter in chapterfiles.keys():
    sampledict[chapter] = {}
    for file in chapterfiles[chapter]:
        openfile = pandas.read_csv(usgs_splib07.open(file),delim_whitespace=False)
        
        instrument_index = len(openfile.keys()[0].split())-2
        instrument_name = openfile.keys()[0].split()[instrument_index]
        instrument_list.append(instrument_name)
        
        if 'ASD' in instrument_name:
            wavelength = wavelength_asdfr
        elif 'AVIRIS' in instrument_name:
            wavelength = wavelength_aviris
        elif 'BECK' in instrument_name:
            wavelength = wavelength_beck
        elif 'NIC4' in instrument_name:
            wavelength = wavelength_nic4
                
        
        sample_name = ' '.join(openfile.keys()[0].split()[2:instrument_index])

        openfile.columns.values[0] = 'rsr'

        rsr = openfile

        
        sampledict[chapter][sample_name] = pandas.DataFrame({'wavelength': wavelength.squeeze(), 'rsr': rsr.squeeze()})
        sampledict[chapter][sample_name] = sampledict[chapter][sample_name].where(sampledict[chapter][sample_name]['rsr'] != -1.23e+34)

In [ ]:
# TODO Enable instrument specific discrimination
instrument_list = set(instrument_list)
instrument_list
#for instrument in instrument_list:

In [ ]:
from ipywidgets import interact, Dropdown

sensorW = Dropdown(options = sensordict.keys())
chapterW = Dropdown(options = sampledict.keys())

sampleW = Dropdown(options = sampledict[chapterW.value].keys())

@interact(sensor = sensorW, chapter = chapterW, sample = sampleW)

def print_sample(sensor, chapter, sample):
    sensorW.options = sensordict.keys()
    chapterW.options = sampledict.keys()
    sampleW.options = sampledict[chapter].keys()
    speclib.plotallbandsplussample(sensordict, sampledict, sensor, chapter, sample)
    return(sensor,  chapter, sample)

# NASA-sensor-plot-emulation

## Sensor Intercomparison

In [ ]:
# Import sensor dict from JSON
# TODO integrate into sensordict

def js_r(fp: str):
    with open(fp) as f_in:
        return json.load(f_in)

sensor_print_dict = js_r('sensor_plot_spec.json')

In [ ]:
sensorlistW = SelectMultiple(options = sensordict.keys())
#sensorlistW = SelectMultiple(options = plotdict.keys())
startW = Dropdown(options = ['Pause', 'Go'])

plotlimitminW = Text(
    value='350',
    description='PlotLimitMin Value:',style={'description_width': 'initial'},
    disabled=False   
)    
plotlimitmaxW = Text(
    value='2500',
    description='PlotLimitMax Value:',
    disabled=False   
) 
plotlimitminW2 = Text(
    value='7500',
    description='PlotLimitMin Value:',
    disabled=False   
)    
plotlimitmaxW2 = Text(
    value='14000',
    description='PlotLimitMax Value:',
    disabled=False   
) 
plotticklistW = Text(
    value='400,900,1500,1900,2400',
    description='PlotTickLlistX List:',
    disabled=False   
)
plotticklistW2 = Text(
    value='8000,9000,10000,11000,12000,13000',
    description='PlotTickLlistX List:',
    disabled=False   
)

titleW = Text(
    value='Sensor bands at Full Width Half Maximum',
    placeholder='Plot label',
    description='String:',
    disabled=False   
)

heightW = Text(
    value='4',
    description='Height of sensor band block',
    disabled=False   
)

figureHeightW = Text(
    value='10',
    description='Figure Y size',
    disabled=False   
)

figureWidthW = Text(
    value='20',
    description='Figure X size',
    disabled=False   
)
lineWidthW = Text(
    value='0.5',
    description='Box line width',
    disabled=False   
)
# Create a LUT to be used to determine the colour of a block in the chart based on proximity of centre wavelength to the LUT value
colourlut = { 1: {'hexcolour': '#008FA2'}
              , #Pan
              420: #coastal
              {'hexcolour':'#72A1C7'}
             ,
              478: 
               {'hexcolour':'#00648D'}
              , #blue
              590:
               {'hexcolour':'#4B9F5F'}
              , #green
              660:
               {'hexcolour':'#C21E37'}
              , #red
              740:
             { 'hexcolour': '#7D2652'}
             , # dark red
              850:
               {'hexcolour': '#C5A2BD'}
              , #NIR
              1650:
               {'hexcolour': '#D39979'}
              , #SWIR1
              2200:
               {'hexcolour': '#999C96'}
              , #SWIR2
              8500:
                   {'hexcolour':'#BC7A82'}
              , #TIR
             }

@interact(sensorlist = sensorlistW, title=titleW, plotlimitmin=plotlimitminW, plotlimitmax=plotlimitmaxW, plotticklist=plotticklistW,plotlimitmin2=plotlimitminW2,
          plotlimitmax2=plotlimitmaxW2, plotticklist2=plotticklistW2, start = startW, height=heightW, figureWidth=figureWidthW, figureHeight=figureHeightW, lineWidth=lineWidthW)

def print_sample( sensorlist, plotlimitmin, plotlimitmax, plotticklist ,plotlimitmin2, plotlimitmax2, plotticklist2, start, title, height, figureWidth, figureHeight, lineWidth):
    titleW = title
    sensorlistW = list(sensordict.keys())
    plotlimitminW = plotlimitmin
    plotlimitmaxW = plotlimitmax
    plotticklistW = plotticklist
    plotlimitminW2 = plotlimitmin2
    plotlimitmaxW2 = plotlimitmax2
    plotticklistW2 = plotticklist2
    heightW = height
    figureWidthW = figureWidth
    figureHeightW = figureHeight
    lineWidthW = lineWidth
    startW.options
    if start == 'Go':
        plotticklist = [int(i) for i in plotticklist.split(',')]

        #Reorder the sensorlist by launch date
        ordered_sensordict = {}
        launched = []
        for sensor in sensorlist:
            year = sensor_print_dict[sensor]['launched']
            launched.append((year, sensor))
        launched = sorted(launched)
        sensorlist = []
        for year, sensor in launched:

            sensorlist.append(sensor)
        
        speclib.plothorizontalallbands( sensordict,  sensorlist, int(plotlimitmin), int(plotlimitmax), plotticklist, int(plotlimitmin2), int(plotlimitmax2), plotticklist2, sensor_print_dict, title, height, figureWidth, figureHeight, colourlut, lineWidth)

# Note
# Rectangles may not display because the Limits are insufficient to enable them to be drawn (i.e. to fix simply increase limits).
# Similarly, if text is overlapping, simply increase the Xlimit